### EGGS Performance Results

In [7]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import cufflinks as cf
import plotly.offline

from IPython.display import display, HTML
from ipywidgets import interactive_output, HBox, VBox, Layout 
from ipywidgets import Dropdown, Checkbox, ToggleButtons, SelectionRangeSlider, SelectionSlider
import ipywidgets as widgets

cf.go_offline()
pd.set_option('display.max_columns', 100)

data_dir = '../../'
df = pd.read_csv(os.path.join(data_dir, 'results.csv'))
df.head()

df[df['feature_type'] == 'full']

,dataset,rs,base_estimator,feature_type,test_type,sgl_method,sgl_stacks,pgm,auc,ap
25,youtube,1,lr,full,limited,None,0,None,0.853292,0.572996
26,youtube,1,lr,full,limited,None,0,mrf,0.853373,0.600856
27,youtube,1,lr,full,limited,holdout,1,None,0.821024,0.537164
28,youtube,1,lr,full,limited,holdout,1,psl,0.815474,0.508118
29,youtube,1,lr,full,limited,holdout,1,mrf,0.834772,0.566076
30,youtube,1,lr,full,limited,holdout,2,None,0.810778,0.564435
31,youtube,1,lr,full,limited,holdout,2,psl,0.819734,0.547290
32,youtube,1,lr,full,limited,holdout,2,mrf,0.830723,0.582178
33,youtube,1,lr,full,limited,cv,1,None,0.860540,0.581715
34,youtube,1,lr,full,limited,cv,1,mrf,0.858201,0.607036


In [10]:
# plot options
x = Dropdown(description='x', options=['topd', 'max_depth'])
metric = SelectionSlider(description='Metric', options=['auc', 'ap'], value='auc')
categories = Dropdown(description='Categories', options=['lmbda', 'topd', 'max_depth'])
# naive = Checkbox(description='naive', value=True)
# exact = Checkbox(description='exact (lambda=-1)', value=True)

# experiment options
dataset = Dropdown(description='Dataset', options=['youtube', 'twitter', 'soundcloud'], value='youtube')
feature_type = ToggleButtons(description='Feature set', options=['full', 'limited'], value='limited')
test_type = ToggleButtons(description='Test', options=['full', 'inductive'], value='full')
base_estimator = ToggleButtons(description='Criterion', options=['lr', 'lgb'], value='lr')
rs = SelectionSlider(description='rs', options=[1], value=1)

# hyperparameter options
sgl_method = SelectionSlider(description='SGL method', options=['None', 'holdout', 'cv'], value='None')
sgl_stacks = SelectionRangeSlider(description='SGL stacks', options=[0, 1, 2], index=(0, 0))
pgm = SelectionSlider(description='PGM', options=['None', 'PSL', 'MRF'], value='None')

# create ui
box_1 = VBox([x, categories, metric, rs])
box_2 = VBox([dataset, feature_type, test_type, base_estimator])
box_3 = VBox([sgl_method, sgl_stacks, pgm])
ui = HBox([box_2, box_3, box_1])

# plot graphs
def f(x, metric, categories,
      dataset, feature_type, test_type, base_estimator, rs,
      sgl_method, sgl_stacks, pgm):
    
    # filter results
    temp = df.copy()
    temp = temp[temp['dataset'] == dataset]
    temp = temp[temp['feature_type'] == feature_type]
    temp = temp[temp['test_type'] == test_type]
    temp = temp[temp['base_estimator'] == base_estimator]
    temp = temp[temp['rs'] == rs]
    
    temp = temp[temp['sgl_method'] == sgl_method]
    temp = temp[(temp['sgl_stacks'] >= sgl_stacks[0]) & (temp['sgl_stacks'] <= sgl_stacks[1])]
    temp = temp[temp['pgm'] == pgm]
    print(temp)

    # plot results
#     if len(temp) > 0:
        
#         y = ['amortized', 'speedup_vs_naive', metric, '{}_diff_avg'.format(metric),
#             'percent_complete', 'num_retrains']
#         titles = ['Absolute efficiency (lower -> better)', 'Relative efficiency (higher -> better)',
#                   'Absolute utility (higher -> better)', 'Relative utility (lower -> better)',
#                   'Completed deletions', 'Retrains']
        
#         if retrain_depth:
#             y[-2] = 'avg_retrain_depth'
#             titles[-2] = 'Retrain depth'

#         nrows, ncols = 2, 3
#         fig = plt.figure(figsize=(15, 4.5), constrained_layout=True)
#         gs = gridspec.GridSpec(nrows=nrows, ncols=ncols, figure=fig)
#         axs = [fig.add_subplot(gs[j, i]) for i in range(ncols) for j in range(nrows)]

#         for i, ax in enumerate(axs):
#             ax.set_xlabel(x)
#             ax.set_ylabel(y[i])
#             ax.set_title(titles[i])

#         if x_scale:
#             for ax in axs:
#                 ax.set_xscale('log')

#         colors = ['green', 'orange', 'magenta', 'purple', 'cyan', 'blue']
#         colors2 = ['brown', 'black', 'red']
#         lines2 = ['--', ':', '-.']
#         labels2 = ['naive', 'exact', 'random']
#         markersize = 120
#         alpha = 0.75

#         for i, (label, gf) in enumerate(temp.groupby(categories)):
#             for j, ax in enumerate(axs):
#                 ax.plot(gf[x], gf[y[j]], label=label, color=colors[i], alpha=alpha, marker='o')

#         if naive and len(naive_df) > 0:
#             for i in [0, 1]:
#                 if x == 'topd':
#                     axs[i].axhline(naive_df.iloc[0][y[i]], label=labels2[0], color=colors2[0],
#                                    alpha=alpha, marker='o', linestyle=lines2[0])
#                 else:
#                     axs[i].plot(naive_df[x], naive_df[y[i]], label='naive', color='brown',
#                                 alpha=alpha, marker='o', linestyle='--')
            
#         if exact and len(exact_df) > 0:
#             for i, ax in enumerate(axs):
#                 if x == 'topd':
#                     axs[i].axhline(exact_df.iloc[0][y[i]], label=labels2[1], color=colors2[1],
#                                    alpha=alpha, marker='o', linestyle=lines2[1])
#                 else:
#                     axs[i].plot(exact_df[x], exact_df[y[i]], label='exact', color='black',
#                                 alpha=alpha, marker='o', linestyle=':')
            
#         axs[0].legend(title=categories)
    
#     else:
#         plt.clf()

# # dictionaries
# dataset_dict = {'surgical': (100, (4, 4), 'acc'), 'adult': (10, (4, 4), 'acc'), 'bank_marketing': (100, (3, 3), 'auc'),
#                 'flight_delays': (250, (4, 4), 'auc'), 'diabetes': (250, (4, 4), 'acc'), 'olympics': (250, (4, 4), 'auc'),
#                 'census': (250, (4, 4), 'auc'), 'credit_card': (250, (4, 4), 'ap'), 'synthetic': (250, (4, 4), 'acc'),
#                 'higgs': (100, (3, 3), 'acc')}

# def update_trees(*args):
#     trees.value = dataset_dict[dataset.value][0]
#     metric.value = dataset_dict[dataset.value][2]
#     if x.value == 'topd':
#         depth.index = dataset_dict[dataset.value][1]

# def update_categories(*args):
#     if x.value == 'max_depth':
#         categories.value = 'topd'
#         depth.index = (0, 4)
    
#     elif x.value == 'topd':
#         categories.value = 'lmbda'
#         depth.index = dataset_dict[dataset.value][1]

# dataset.observe(update_trees, names='value')
# x.observe(update_categories, names='value')

out = interactive_output(f, {'x': x, 'metric': metric, 'categories': categories,
                             'dataset': dataset, 'feature_type': feature_type, 'test_type': test_type,
                             'base_estimator': base_estimator, 'rs': rs,
                             'sgl_method': sgl_method, 'sgl_stacks': sgl_stacks, 'pgm': pgm})
display(ui, out)

Output()